In [1]:
ModelsDir <- '/home/kate/Research/Property/Models/'
DataDir <- '/home/kate/Research/Property/Data/'
ModelName <- "wc_poissonglm_f_ecy"
UseSavedIfExists <- TRUE

In [2]:
source('/home/kate/code/Utils/MyFunctions.R')

In [3]:
training_dataset <- read.csv(paste(DataDir,"property_wcf_training.csv", sep = ""), header=TRUE)
testing_dataset <- read.csv(paste(DataDir,"property_wcf_testing.csv", sep = ""), header=TRUE)
prediction_dataset <- read.csv(paste(DataDir,"property_water_claims_non_cat_fs.csv", sep = ""), header=TRUE)

In [4]:
formula <- cova_ic_nc_water ~ landlordind + log_ecy + log_sqft +      log_yearbuilt + roofcd + usagetype + cova_deductible + log_water_risk_3_blk

In [5]:
kfold <- 5

In [6]:
models_attr <- data.frame()
models_coef <- data.frame()
training_dataset$poissonglm_f_ecy_cv <- 0
testing_dataset$poissonglm_f_ecy_cv <- 0
prediction_dataset$poissonglm_f_ecy_cv <- 0
for (fold in 1:kfold-1){
  training_dataset[paste("poissonglm_f_ecy_",fold,sep="")] <- 0 
  testing_dataset[paste("poissonglm_f_ecy_",fold,sep="")] <- 0  
  prediction_dataset[paste("poissonglm_f_ecy_",fold,sep="")] <- 0    
    }
# 
#CV-Folds
AIC_l <- list();
BIC_l <- list();
logLik_l <- list();
coef_l <- list();
folds_l <- list() 
i <- 1
for (fold in 1:kfold-1){
    print(paste("Fold ",fold," of ",kfold," currently processing...",sep=""))
    training_dataset_fold <- training_dataset[training_dataset[paste("fold",fold,sep="_")] > 0,]
    validation_dataset <- training_dataset[training_dataset[paste("fold",fold,sep="_")] == 0,]
    #Modeling
    ModelFile <- paste(ModelsDir,ModelName,"_",fold,".rds",sep="")
    if(file.exists(ModelFile) && UseSavedIfExists){
        poissonglm <- readRDS(ModelFile)
    } else {
        poissonglm <- glm(formula, family = "poisson", data = training_dataset_fold)
        saveRDS(poissonglm, ModelFile)             
    }#if modeling 
    #Training Prediction
    pred_train <- predict(poissonglm,training_dataset, type="response")
    training_dataset[paste("poissonglm_f_ecy_",fold,sep="")] <- pred_train
    training_dataset$poissonglm_f_ecy_cv <- training_dataset$poissonglm_f_ecy_cv + pred_train/kfold
    #Testing Prediction
    pred_test <- predict(poissonglm,testing_dataset, type="response")
    testing_dataset$poissonglm_f_ecy_cv <- testing_dataset$poissonglm_f_ecy_cv + pred_test/kfold 
    testing_dataset[paste("poissonglm_f_ecy_",fold,sep="")] <- pred_test
    #Prediction
    pred <- predict(poissonglm,prediction_dataset, type="response")
    prediction_dataset$poissonglm_f_ecy_cv <- prediction_dataset$poissonglm_f_ecy_cv + pred/kfold 
    prediction_dataset[paste("poissonglm_f_ecy_",fold,sep="")] <- pred
    #Model attributes
    folds_l[[i]] <- i
    AIC_l[[i]] <- AIC(poissonglm)
    BIC_l[[i]] <- BIC(poissonglm)  
    logLik_l[[i]] <- logLik(poissonglm) 
    #Model coeficients
    model_coef_fold <- data.frame(
        fold = fold+1,
        pred_name = summary(poissonglm)$coefficients[,0],
        estimate =  summary(poissonglm)$coefficients[,1],
        StdError =  summary(poissonglm)$coefficients[,2],  
        zValue =   summary(poissonglm)$coefficients[,3],     
        valuePr =   summary(poissonglm)$coefficients[,4]    
    )  
    #pred_name is an index, row name, convert it into column
    model_coef_fold <- cbind(name = rownames(model_coef_fold), model_coef_fold)
    rownames(model_coef_fold) <- 1:nrow(model_coef_fold) 
    #main table
    models_coef <- rbind(models_coef,model_coef_fold)         
    i <- i + 1
    }#folds loop   
    #Model attributes data frame
    models_fold_attr <- data.frame( 
        fold = unlist(folds_l),
        AIC = unlist(AIC_l),
        BIC = unlist(BIC_l),   
        logLik = unlist(logLik_l)
    )
    models_attr <- rbind(models_attr,models_fold_attr)                 

[1] "Fold 0 of 5 currently processing..."
[1] "Fold 1 of 5 currently processing..."
[1] "Fold 2 of 5 currently processing..."
[1] "Fold 3 of 5 currently processing..."
[1] "Fold 4 of 5 currently processing..."


In [7]:
head(models_attr)

fold,AIC,BIC,logLik
1,91363.21,91591.39,-45662.61
2,91253.60,91481.77,-45607.80
3,91224.78,91452.96,-45593.39
4,91144.17,91372.35,-45553.09
5,91217.95,91446.13,-45589.97


In [8]:
write.table(models_attr,paste(ModelsDir,ModelName,"_attr.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)

In [9]:
head(models_coef)

name,fold,estimate,StdError,zValue,valuePr
(Intercept),1,-119.49280887,9.53107890,-12.5371755,4.673579e-36
landlordind,1,-0.24359037,0.07314753,-3.3301243,8.680723e-04
log_ecy,1,0.93725121,0.02137019,43.8578789,0.000000e+00
log_sqft,1,0.57670349,0.03254902,17.7179985,3.045572e-70
log_yearbuilt,1,13.96155322,1.25969306,11.0832977,1.511990e-28
roofcdMEMBRANE,1,0.04654997,0.10265165,0.4534752,6.502066e-01


In [10]:
write.table(models_coef,paste(ModelsDir,ModelName,"_coef.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)

Normalized Weighted gini Training

In [11]:
NormalizedWeightedGini(training_dataset$cova_ic_nc_water,training_dataset$poissonglm_f_ecy_cv,training_dataset$ecy)

[1] 0.3802661

Normalized Weighted gini Testing

In [12]:
NormalizedWeightedGini(testing_dataset$cova_ic_nc_water,testing_dataset$poissonglm_f_ecy_cv,testing_dataset$ecy)

[1] 0.4041194

In [13]:
write.table(training_dataset,paste(DataDir,"property_wcf_training.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)
write.table(testing_dataset,paste(DataDir,"property_wcf_testing.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)
write.table(prediction_dataset,paste(DataDir,"property_water_claims_non_cat_fs.csv", sep = ""), sep=",",  col.names=TRUE, row.names = FALSE)